In [ ]:
"""PARAMETERS TO CAPTURE ON STDIN VIA PAPERMILL...."""
#config_path = 'configs/species_experiments/Escherichia_coli_lysate.yaml'
#config_path = 'configs/global_loss_composed_globalAvgtm/global_allmeltome.yaml'
#config_path = 'configs/global_loss_composed_globalAvgtm/global_allmeltome_cleaned.yaml'
#config_path = 'configs/species_experiments_loss_composed_globalAvgtm_cleanup/loss2/Arabidopsis_thaliana_seedling_lysate.yaml'
#config_path = "configs/species_experiments_loss_composed_globalAvgtm/loss2/Geobacillus_stearothermophilus_NCA26_lysate.yaml"

#config_path = "configs/tmp/Arabidopsis_thaliana_seedling_lysate_rankn.yaml"
#config_path = "configs/tmp/Arabidopsis_thaliana_seedling_lysate_rankn_l2_l1_MSE.yaml"
#config_path = "configs/tmp/Arabidopsis_thaliana_seedling_lysate_rankn_l2_l1_MSE_5_percent.yaml"
#config_path = "configs/tmp/Arabidopsis_thaliana_seedling_lysate_rankn_l2_l1_MSE_normal_noise_persample.yaml"
#config_path = "configs/tmp/Arabidopsis_thaliana_seedling_lysate_rankn_l2_l1_MSE_normal_noise_persample_diagonal.yaml"


#config_path = "configs/tmp/Arabidopsis_thaliana_seedling_lysate_rankn_tmp.yaml"
#config_path = "configs/tmp/Arabidopsis_thaliana_seedling_lysate_Bias_Global.yaml"
#config_path = "configs/tmp/Arabidopsis_thaliana_seedling_lysate_Bias_batch.yaml"
#config_path = "configs/tmp/Arabidopsis_thaliana_seedling_lysate_MSE.yaml"
#config_path = "configs/tmp/Arabidopsis_thaliana_seedling_lysate_Balancing_Samples.yaml"
#config_path = 'configs/experiments_10_05_2024/global/global_allmeltome_biasg.yaml'
#config_path = 'configs/experiments_10_05_2024/species_specific/biasg/Caenorhabditis_elegans_lysate.yaml'
#config_path = 'configs/experiments_10_05_2024/global_vs_species/biasg/Caenorhabditis_elegans_lysate.yaml'
#config_path = 'configs/experiments_10_05_2024/global_vs_species/rankN/K562.yaml'
config_path = 'configs/experiments_10_05_2024/species_specific/balanceMSE/Saccharomyces_cerevisiae_lysate.yaml'

In [ ]:
import yaml

print(f"the value of path captured is: {config_path}")

with open(config_path, 'r') as file:
    config = yaml.safe_load(file)
    
#print(config)
#print(f"the value of path captured is: {config_path}")

In [3]:
#import ipdb; ipdb.set_trace()

path_progresESM    = config['initial_paths']['path_progresESM']
checkpoint_pathESM = config['initial_paths']['checkpoint_pathESM']

path_progresPiFold    = config['initial_paths']['path_progresPiFold']
checkpoint_pathPiFold = config['initial_paths']['checkpoint_pathPiFold']


path_progresMix= config['initial_paths']['path_progresMix']
#checkpoint_pathMix='../checkpoints/LA_MLP_TaskVectorBias_Merged_allnet_1_10_2023_finetunning_NoMLPincNoBN_lrx00001_lossmod/'
checkpoint_pathMix= config['initial_paths']['checkpoint_pathMix']

#import ipdb; ipdb.set_trace()

organism = config['organism']['organism']
keyword = config['organism']['keyword']

#ESM_source = 'esm2_embeddings'
ESM_source = config['initial_paths']['ESM_source']
PiFold_source = config['initial_paths']['PiFold_source']

prepro_train_emb_ESM = config['initial_paths']['prepro_train_emb_ESM']
prepro_test_emb_ESM  = config['initial_paths']['prepro_test_emb_ESM']
prepro_val_emb_ESM   = config['initial_paths']['prepro_val_emb_ESM']

prepro_train_emb_PiFold = config['initial_paths']['prepro_train_emb_PiFold']
prepro_test_emb_PiFold  = config['initial_paths']['prepro_test_emb_PiFold']
prepro_val_emb_PiFold   = config['initial_paths']['prepro_val_emb_PiFold']

In [4]:
import __init__

from src.utilities import train_test_validation_splits, \
                            prepare_train_test_val_seqs_by_batches, \
                            tensor2dataloader, load_full_meltome_FLIP_db, \
                            seek_UniprotID_association_Meltome_prots,\
                            download_UniprotID_Alphafold_Structures,\
                            get_guided_encoder_output,\
                            ESM2_IF_repr
                            
import pandas as pd
import numpy as np
    
import ipdb
path_meltome = config['meltome']['path_meltome']
complete_meltome_db = config['meltome']['complete_meltome_db']

full_meltome_db = load_full_meltome_FLIP_db(complete_meltome_db)

splits_meltome = pd.read_csv(path_meltome, sep=',')

#import ipdb; ipdb.set_trace()

splits_meltome = seek_UniprotID_association_Meltome_prots(splits_meltome, full_meltome_db)
split_meltome = splits_meltome[splits_meltome['sequence'].str.len()>= 50]
#import ipdb; ipdb.set_trace()
'''------------------------------------------------------------------------------------------------------------------------------'''
'''------------------------------------------------------------------------------------------------------------------------------'''
'''                      JUST FILTERING TO TAKE BACTERIA AND IGNORE EUKARIOTIC SAMPLES (HUMAN, PLANTS, CANCER CELLS, ETC)        '''

list_bacteria = [ "Caenorhabditis_elegans_lysate", "Saccharomyces_cerevisiae_lysate", "Escherichia_coli_lysate", 
                 "Escherichia_coli_cells", "Bacillus_subtilis_168_lysate_R1", "Oleispira_antarctica_RB-8_lysate_R1", 
                 "Picrophilus_torridus_DSM9790_lysate", "Thermus_thermophilus_HB27_cells", 
                 "Geobacillus_stearothermophilus_NCA26_lysate", "Thermus_thermophilus_HB27_lysate"]
#splits_meltome = splits_meltome[ splits_meltome.organism.isin( list_bacteria ) ]

'''------------------------------------------------------------------------------------------------------------------------------'''
'''------------------------------------------------------------------------------------------------------------------------------'''

train, val, test = train_test_validation_splits(splits_meltome)

print("Done")



Done


In [3]:
import torch
import src.IF_PiFold_embeddings  as PiFoldconf 
#from src.IF_PiFold_embeddings import PiFold_setup


args = PiFoldconf.get_parser()
exp = PiFoldconf.PiFold_setup(args)
svpath = config['Pifold_pretrain']['svpath']
exp.method.load_state_dict(torch.load(svpath+'checkpoint.pth'))


Use GPU: cuda:0



device: 	cuda	
display_step: 	10	
res_dir: 	results	
ex_name: 	ProDesign	
use_gpu: 	True	
gpu: 	0	
seed: 	111	
data_name: 	CATH	
data_root: 	data/	
batch_size: 	8	
num_workers: 	8	
method: 	ProDesign	
config_file: 	None	
hidden_dim: 	128	
node_features: 	128	
edge_features: 	128	
k_neighbors: 	30	
dropout: 	0.1	
num_encoder_layers: 	10	
epoch: 	100	
log_step: 	1	
lr: 	0.001	
patience: 	100	
updating_edges: 	4	
node_dist: 	1	
node_angle: 	1	
node_direct: 	1	
edge_dist: 	1	
edge_angle: 	1	
edge_direct: 	1	
virtual_num: 	3	


<All keys matched successfully>

In [5]:
from tqdm.auto import tqdm
import torch
from src.ESM2embeddings import ESM2embeddings
from src.ESM2_IF1_embeddings import ESM2_IF1_embeddings
from src.utilities import train_test_validation_splits, prepare_train_test_val_seqs_by_batches, tensor2dataloader,read_and_plot_loss_curves, cleanup_by_organism_and_correspondence_idx
#from src.NeuralArchitectures import regressionHead

from src.AdaptiveDataset import AdaptiveDataset, Species_Balancing_BatchSampler

structuredir =  config['structures_path']['structuredir']
meltome_struct_dir_labels =  config['structures_path']['meltome_struct_dir_labels']

import ipdb; ipdb.set_trace()
import os 
if not os.listdir(structuredir) and not os.path.isfile(meltome_struct_dir_labels+'Meltome_Splits_FLIPS_train.txt')\
                                and not os.path.isfile(meltome_struct_dir_labels+'Meltome_Splits_FLIPS_test.txt')\
                                and not os.path.isfile(meltome_struct_dir_labels+'Meltome_Splits_FLIPS_val.txt'): 
    
    print("Finding structures to proteins via Alphafold/PDB dbs..........") 
    
    download_UniprotID_Alphafold_Structures(train, structuredir, meltome_struct_dir_labels, 'Meltome_Splits_FLIPS_train')
    download_UniprotID_Alphafold_Structures(test, structuredir, meltome_struct_dir_labels, 'Meltome_Splits_FLIPS_test')
    download_UniprotID_Alphafold_Structures(val, structuredir, meltome_struct_dir_labels, 'Meltome_Splits_FLIPS_val')


#import ipdb; ipdb.set_trace()
Structinfo_Meltome_Splits_train = pd.read_csv( meltome_struct_dir_labels+'Meltome_Splits_FLIPS_train.txt', sep='\t').set_index(train.index)
Structinfo_Meltome_Splits_test = pd.read_csv( meltome_struct_dir_labels+'Meltome_Splits_FLIPS_test.txt', sep='\t').set_index(test.index)
Structinfo_Meltome_Splits_val = pd.read_csv( meltome_struct_dir_labels+'Meltome_Splits_FLIPS_val.txt', sep='\t').set_index(val.index)


#import ipdb; ipdb.set_trace()

train = pd.concat([train, Structinfo_Meltome_Splits_train], axis=1).query('Source=="alphafold"')
test = pd.concat([test, Structinfo_Meltome_Splits_test], axis=1).query('Source=="alphafold"')
val = pd.concat([val, Structinfo_Meltome_Splits_val], axis=1).query('Source=="alphafold"')

# Copy of complete Meltome datasets for embedding creation in case of being necesary
full_train = train.copy()
full_test = test.copy()
full_val = val.copy()
"""
import ipdb; ipdb.set_trace()
if config['organism']['Global']==True:
    idxdf_idxfile_train, _ = cleanup_by_organism_and_correspondence_idx(train, organism, keyword)
    idxdf_idxfile_test, _ = cleanup_by_organism_and_correspondence_idx(test, organism, keyword)
    idxdf_idxfile_val, _ = cleanup_by_organism_and_correspondence_idx(val, organism, keyword)
    
    if 'umap_clusters_clean_meltome_train' in config['meltome']:
        clusters=pd.read_csv(config['meltome']['umap_clusters_clean_meltome_train'], sep='\t').cluster.tolist()
        train['umap_clusters'] = clusters
        train =  train[train.umap_clusters !=-1]
    
else:
    idxdf_idxfile_train, train = cleanup_by_organism_and_correspondence_idx(train, organism, keyword)
    idxdf_idxfile_test, test = cleanup_by_organism_and_correspondence_idx(test, organism, keyword)
    idxdf_idxfile_val, val = cleanup_by_organism_and_correspondence_idx(val, organism, keyword)
"""

#import ipdb; ipdb.set_trace()
if 'umap_clusters_clean_meltome_train' in config['meltome']:
    clusters=pd.read_csv(config['meltome']['umap_clusters_clean_meltome_train'], sep='\t').cluster.tolist()
    # cluster keys columns are related to the dictionary from cleanup_by_organism_and_correspondence_idx method
    # so if you need a mapping of the sample, you need to use that index to get the proper sampled idx from original
    # pandas dataframe. Recap : idx_numbering -> pandas_idx_original_cleaned_data
    train['umap_clusters'] = clusters

if config['organism']['Global']==True:
    idxdf_idxfile_train, _ = cleanup_by_organism_and_correspondence_idx(train, organism, keyword)
    idxdf_idxfile_test, _ = cleanup_by_organism_and_correspondence_idx(test, organism, keyword)
    idxdf_idxfile_val, _ = cleanup_by_organism_and_correspondence_idx(val, organism, keyword)
    
    if 'umap_clusters_clean_meltome_train' in config['meltome']:
        train =  train[train.umap_clusters !=-1]
    
    ''' In case we want a universal average tm independent from organism - Unlikely'''    
    #train['OGTest_organism']=[train.tm_organism.mean()]*len(train)
    #val['OGTest_organism']=[val.tm_organism.mean()]*len(val)
    train['OGTest_organism'] = train.organism.map( train.groupby('organism').apply(lambda a: a.target.astype(float).mean()) )
    val['OGTest_organism'] = val.organism.map( val.groupby('organism').apply(lambda a: a.target.astype(float).mean()) )
 
else:
    idxdf_idxfile_train, train = cleanup_by_organism_and_correspondence_idx(train, organism, keyword)
    idxdf_idxfile_test, test = cleanup_by_organism_and_correspondence_idx(test, organism, keyword)
    idxdf_idxfile_val, val = cleanup_by_organism_and_correspondence_idx(val, organism, keyword)
    
    if 'umap_clusters_clean_meltome_train' in config['meltome']:
        train =  train[train.umap_clusters !=-1]
    
    #train['OGTest_organism']=[train.tm_organism.mean()]*len(train)
    #val['OGTest_organism']=[val.tm_organism.mean()]*len(val)
    
    train['OGTest_organism']=[train.target.astype(float).mean()]*len(train)
    val['OGTest_organism']=[val.target.astype(float).mean()]*len(val)



'------------------------------------------------------------------------------------------------------------------------------'

In [6]:
train.shape
print(train)
print(train.columns)

                                                sequence            target  \
0      MSGEEEKAADFYVRYYVGHKGKFGHEFLEFEFRPNGSLRYANNSNY...  37.9629473421417   
1      MSMGSDFYLRYYVGHKGKFGHEFLEFEFRPDGKLRYANNSNYKNDV...  54.4253424806097   
2      MRICFLLLAFLVAETFANELTRCCAGGTRHFKNSNTCSSIKSEGTS...  49.4592155176475   
3      MIRVALPTTASAIPRSISTSPGETISKNHEEEVKRVWRKADAVCFD...  42.5931308043435   
4      MNGDWSRAFVLSKVKNLYFFVIIDKGFSAILNDPREPVQVGGFFEV...  37.9994780790578   
...                                                  ...               ...   
27944  MAVVLPAVVEELLSEMAAAVQESARIPDEYLLSLKFLFGSSATQAL...  56.8600005753223   
27945  MLSCNICGETVTSEPDMKAHLIVHMESEIICPFCKLSGVNYDEMCF...  45.1243026020368   
27946  MDLPALLPAPTARGGQHGGGPGPLRRAPAPLGASPARRRLLLVRGP...   47.008000426981   
27947  MPEDQAGAAMEEASPYSLLDICLNFLTTHLEKFCSARQDGTLCLQE...  51.8978809719254   
27948  MAAPRPSPAISVSVSAPAFYAPQKKFGPVVAPKPKVNPFRPGDSEP...  63.5734388197048   

         set  validation  protein_id  \
0      train       Fals

In [5]:

import torch, os, gc
import torch_geometric
import torch_sparse
from torch_geometric.nn import MessagePassing
import warnings
warnings.filterwarnings('ignore')


#import ipdb; ipdb.set_trace()
pt_batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ESM2 = ESM2embeddings(type_embedding = "facebook/esm2_t33_650M_UR50D", device = device, type_tool='FacebookESM2')

dir_pre_generated_embeddings =  config['ESM2embeddings']['dir_pre_generated_embeddings'] 
dir_IFPiFold_Meltome_train = dir_pre_generated_embeddings + PiFold_source + '/Meltome_train/'
dir_IFPiFold_Meltome_test = dir_pre_generated_embeddings +  PiFold_source + '/Meltome_test/'
dir_IFPiFold_Meltome_val = dir_pre_generated_embeddings +  PiFold_source + '/Meltome_val/'


path_train= dir_pre_generated_embeddings+prepro_train_emb_ESM
path_test= dir_pre_generated_embeddings+prepro_test_emb_ESM
path_val= dir_pre_generated_embeddings+prepro_val_emb_ESM

"""
path_esm2_train= dir_pre_generated_embeddings+'esm2_embeddings/train/'
path_esm2_test= dir_pre_generated_embeddings+'esm2_embeddings/test/'
path_esm2_val= dir_pre_generated_embeddings+'esm2_embeddings/val/'
"""
import ipdb; ipdb.set_trace()
path_esm2_train= dir_pre_generated_embeddings+ESM_source+'/Meltome/train/'
path_esm2_test= dir_pre_generated_embeddings+ESM_source+'/Meltome/test/'
path_esm2_val= dir_pre_generated_embeddings+ESM_source+'/Meltome/val/'




def padding_tensor( a, size_end ,val_pad=0):
    a_size = a.shape
    f_size = torch.zeros(size_end[0], size_end[1])
    
    if size_end[0]>=a_size[0]:
        f_size[0:a_size[0]] = a
    else:
        f_size = a[0:size_end[0]]
    return f_size

def ensemble_dataset_from_IF_embeddings(folder_path, max_size=600, dim_IF=128):
    import os
    
    paths = os.listdir(folder_path)
    list_max=[]
    tensor_container = torch.zeros(len(paths),max_size,dim_IF)
    print('Starting load of Inverse Folding Embeddings:  \n')
    
    for i in tqdm(range(0,len(paths))):
        tmp_tensor = torch.load(folder_path+paths[i])
        list_max.append(tmp_tensor.shape[0])
        tmp_tensor = padding_tensor(tmp_tensor, [max_size,dim_IF])
        tensor_container[i]=tmp_tensor
        
    print('maximum structure length in meltome: ' + str(max(list_max)))
    return tensor_container
              
def check_existence_paths_embeddings(folder_path):
    
        if not os.path.exists(folder_path):
            # Create the folder
            os.makedirs(folder_path)
            print(f"Folder '{folder_path}' created.")
        else:
            print(f"Folder '{folder_path}' already exists.")
    
def embedding_builder_ESM2(ESM2, device, train, test, val, path_esm2_train, path_esm2_test, path_esm2_val):
    #import ipdb; ipdb.set_trace()
    #if os.path.isfile(path_train)==False and os.path.isfile(path_test)==False and os.path.isfile(path_val)==False:
    if not os.listdir(path_esm2_train):
        print ("Creating Embeddings...")   
        ESM2.esm2embedding(train, device, layer_index=33,folder_path=path_esm2_train) #esm2embedding(train, model, batch_converter, device, layer_index=33)#BERT.embed_dataset(batch_train)
    if not os.listdir(path_esm2_test):
        ESM2.esm2embedding(test, device, layer_index=33,folder_path=path_esm2_test) #esm2embedding(test, model, batch_converter, device, layer_index=33)
    if not os.listdir(path_esm2_val):
        ESM2.esm2embedding(val, device, layer_index=33,folder_path=path_esm2_val) #esm2embedding(val, model, batch_converter, device, layer_index=33)


#def IFBuilder_sets(IF, train, test, val, path_IFtrain, path_IFtest, path_IFval,
#                   idxdf_idxfile_train,
#                   idxdf_idxfile_test,
#                   idxdf_idxfile_val):
def IFBuilder_sets(IF, train, test, val, path_IFtrain, path_IFtest, path_IFval):
    import ipdb; ipdb.set_trace()

    if not os.listdir( path_IFtrain):
        print ("Creating Embeddings...") 
        IF.IF_repr(train, folder_path=path_IFtrain)#, correspondence_idx_df_file=idxdf_idxfile_train)

    if not os.listdir(path_IFtest):
        IF.IF_repr(test, folder_path=path_IFtest)#, correspondence_idx_df_file=idxdf_idxfile_test)

    if not os.listdir(path_IFval):
        IF.IF_repr(val, folder_path=path_IFval)#, correspondence_idx_df_file=idxdf_idxfile_val)


# 2700 is the maximum size of protein structure extracted from Alphafold DB to Meltome Atlas 

import ipdb; ipdb.set_trace()
check_existence_paths_embeddings(path_esm2_train);check_existence_paths_embeddings(dir_IFPiFold_Meltome_train)
check_existence_paths_embeddings(path_esm2_test); check_existence_paths_embeddings(dir_IFPiFold_Meltome_test)
check_existence_paths_embeddings(path_esm2_val); check_existence_paths_embeddings(dir_IFPiFold_Meltome_val)

embedding_builder_ESM2(ESM2, device, full_train, full_test, full_val,  
                       path_esm2_train, 
                       path_esm2_test, 
                       path_esm2_val)

IFBuilder_sets(exp, full_train, full_test, full_val, 
                dir_IFPiFold_Meltome_train, 
                dir_IFPiFold_Meltome_test, 
                dir_IFPiFold_Meltome_val)#,
                #idxdf_idxfile_train,
                #   idxdf_idxfile_test,
                #   idxdf_idxfile_val)


'\nIF_embeddings_train = ensemble_dataset_from_IF_embeddings(dir_IFPiFold_Meltome_train,max_size=2700)\nIF_embeddings_test = ensemble_dataset_from_IF_embeddings(dir_IFPiFold_Meltome_test, max_size=2700)\nIF_embeddings_val = ensemble_dataset_from_IF_embeddings(dir_IFPiFold_Meltome_val, max_size=2700)\n'

In [6]:
#_, _, _ = embedding_builder_ESM2(ESM2, device, train, test, val,  path_train, path_test, path_val)
''' Now we make the dataloader for each partition of each training set for cross validation'''

#import ipdb; ipdb.set_trace()
#partitions_crossval_train = leave_one_partition_out(train, 5)
def Sampler_definition(config, df, batch_size):
    #import ipdb; ipdb.set_trace()

    if 'sampler_balance' in config:
        return Species_Balancing_BatchSampler(df, batch_size, shuffle=True, tag_search='organism' )
    else:
        return None

#Sampler_Train = Sampler_definition(config, train, batch_size=150)
#Sampler_Val = Sampler_definition(config, val, batch_size=150)
#Sampler_Test = Sampler_definition(config, test, batch_size=150)
#import ipdb; ipdb.set_trace()

"""
def parameters_launcher(config, dataset, df, batch_size, shuffle):
    # This method was made since the batchsample option is mutually exclusive with batch_size,
    # shuffle, etc and it implies to modify the way to send the parameters, at least in the 
    # specifical case of balancing classes via batchsampler.
    sampler = Sampler_definition(config, df, batch_size)
    
    if 'sampler_balance' not in config:    
        
        return {'dataset':dataset, 'batch_size':batch_size, 'shuffle':shuffle,  'batch_sampler':sampler}
    
    else:
        return {'dataset':dataset, 'batch_sampler':sampler}
    
train_embedding_esm2 = torch.utils.data.DataLoader( 
                                                    **parameters_launcher(
                                                        config,
                                                        AdaptiveDataset(train, path_esm2_train, dir_IFPiFold_Meltome_train, 
                                                        emb_ensemble = 'single', type_emb='esm2',
                                                        correspondence_idx = idxdf_idxfile_train ) , train,
                                                        batch_size=150, shuffle=True
                                                   ) )
test_embedding_esm2 = torch.utils.data.DataLoader(
                                                    **parameters_launcher(
                                                        config, 
                                                        AdaptiveDataset(test, path_esm2_test, dir_IFPiFold_Meltome_test, 
                                                        emb_ensemble = 'single', type_emb='esm2',
                                                        correspondence_idx = idxdf_idxfile_test) , test,
                                                        batch_size=150, shuffle=True
                                                   ) )
val_embedding_esm2 = torch.utils.data.DataLoader( 
                                                    **parameters_launcher(
                                                        config, AdaptiveDataset(val, path_esm2_val, dir_IFPiFold_Meltome_val, 
                                                        emb_ensemble = 'single', type_emb='esm2',
                                                        correspondence_idx = idxdf_idxfile_val) , val,
                                                        batch_size=150, shuffle=True) )

"""
def parameters_launcher(config, df, path_esm2, dir_IFPiFold_Meltome, batch_size, shuffle,
                        emb_ensemble, type_emb,
                        correspondence_idx ):
    # This method was made since the batchsample option is mutually exclusive with batch_size,
    # shuffle, etc and it implies to modify the way to send the parameters, at least in the 
    # specifical case of balancing classes via batchsampler.
    sampler = Sampler_definition(config, df, batch_size)
    
    
    #import ipdb; ipdb.set_trace()
    
    if 'sampler_balance' not in config:
        
        dataset = AdaptiveDataset(df, path_esm2, dir_IFPiFold_Meltome, 
            emb_ensemble = emb_ensemble, type_emb=type_emb,
            correspondence_idx = correspondence_idx )
        
        return {'dataset':dataset, 'batch_size':batch_size, 'shuffle':shuffle,  'batch_sampler':sampler}
    
    else:
        
        dataset = AdaptiveDataset(df, path_esm2, dir_IFPiFold_Meltome, 
            emb_ensemble = emb_ensemble, type_emb=type_emb,
            correspondence_idx = correspondence_idx, sampler_use=True )
            
        return {'dataset':dataset, 'batch_sampler':sampler}


#import ipdb; ipdb.set_trace()

train_embedding_esm2 = torch.utils.data.DataLoader(
                                                    **parameters_launcher(
                                                        config,
                                                        train, path_esm2_train, dir_IFPiFold_Meltome_train,
                                                        batch_size=150, shuffle=True, 
                                                        emb_ensemble = 'single', type_emb='esm2', 
                                                        correspondence_idx = idxdf_idxfile_train
                                                   ) )
test_embedding_esm2 = torch.utils.data.DataLoader(
                                                    **parameters_launcher(
                                                        config, 
                                                        test, path_esm2_test, dir_IFPiFold_Meltome_test, 
                                                        batch_size=150, shuffle=True,
                                                        emb_ensemble = 'single', type_emb='esm2',
                                                        correspondence_idx = idxdf_idxfile_test
                                                   ) )
val_embedding_esm2 = torch.utils.data.DataLoader( 
                                                    **parameters_launcher(
                                                        config, 
                                                        val, path_esm2_val, dir_IFPiFold_Meltome_val, 
                                                        batch_size=150, shuffle=True,
                                                        emb_ensemble = 'single', type_emb='esm2',
                                                        correspondence_idx = idxdf_idxfile_val
                                                        ) )



'''------------------------------------------------------------------------------------------------------'''
'''------------------------------------------------------------------------------------------------------'''

train_embedding_PiFold = torch.utils.data.DataLoader(
                                                    **parameters_launcher(
                                                        config,
                                                        train, path_esm2_train, dir_IFPiFold_Meltome_train,
                                                        batch_size=150, shuffle=True, 
                                                        emb_ensemble = 'single', type_emb='PiFold', 
                                                        correspondence_idx = idxdf_idxfile_train
                                                   ) )
test_embedding_PiFold = torch.utils.data.DataLoader(
                                                    **parameters_launcher(
                                                        config, 
                                                        test, path_esm2_test, dir_IFPiFold_Meltome_test, 
                                                        batch_size=150, shuffle=True,
                                                        emb_ensemble = 'single', type_emb='PiFold',
                                                        correspondence_idx = idxdf_idxfile_test
                                                   ) )
val_embedding_PiFold = torch.utils.data.DataLoader( 
                                                    **parameters_launcher(
                                                        config, 
                                                        val, path_esm2_val, dir_IFPiFold_Meltome_val, 
                                                        batch_size=150, shuffle=True,
                                                        emb_ensemble = 'single', type_emb='PiFold',
                                                        correspondence_idx = idxdf_idxfile_val
                                                        ) )

'''------------------------------------------------------------------------------------------------------'''
'''------------------------------------------------------------------------------------------------------'''
train_embedding_merged = torch.utils.data.DataLoader(
                                                    **parameters_launcher(
                                                        config,
                                                        train, path_esm2_train, dir_IFPiFold_Meltome_train,
                                                        batch_size=150, shuffle=True, 
                                                        emb_ensemble = 'merged', type_emb='both', 
                                                        correspondence_idx = idxdf_idxfile_train
                                                   ) )
test_embedding_merged = torch.utils.data.DataLoader(
                                                    **parameters_launcher(
                                                        config, 
                                                        test, path_esm2_test, dir_IFPiFold_Meltome_test, 
                                                        batch_size=150, shuffle=True,
                                                        emb_ensemble = 'merged', type_emb='both',
                                                        correspondence_idx = idxdf_idxfile_test
                                                   ) )
val_embedding_merged = torch.utils.data.DataLoader( 
                                                    **parameters_launcher(
                                                        config, 
                                                        val, path_esm2_val, dir_IFPiFold_Meltome_val, 
                                                        batch_size=150, shuffle=True,
                                                        emb_ensemble = 'merged', type_emb='both',
                                                        correspondence_idx = idxdf_idxfile_val
                                                        ) )
"""
train_embedding_PiFold = torch.utils.data.DataLoader( AdaptiveDataset(train, path_esm2_train, dir_IFPiFold_Meltome_train, 
                                                    emb_ensemble = 'single', type_emb='PiFold',
                                                   correspondence_idx = idxdf_idxfile_train) , batch_size=150, shuffle=True,
                                                   batch_sampler = Sampler_Train)   
test_embedding_PiFold = torch.utils.data.DataLoader( AdaptiveDataset(test, path_esm2_test, dir_IFPiFold_Meltome_test,
                                                    emb_ensemble = 'single', type_emb='PiFold',
                                                   correspondence_idx = idxdf_idxfile_test) , batch_size=150, shuffle=True,
                                                   batch_sampler = Sampler_Test)
val_embedding_PiFold = torch.utils.data.DataLoader( AdaptiveDataset(val, path_esm2_val, dir_IFPiFold_Meltome_val,
                                                    emb_ensemble = 'single', type_emb='PiFold',
                                                   correspondence_idx = idxdf_idxfile_val) , batch_size=150, shuffle=True,
                                                   batch_sampler = Sampler_Val)



train_embedding_merged = torch.utils.data.DataLoader( AdaptiveDataset(train, path_esm2_train, dir_IFPiFold_Meltome_train, 
                                                    emb_ensemble = 'merged', type_emb='both',
                                                   correspondence_idx = idxdf_idxfile_train) , batch_size=150, shuffle=True,
                                                   batch_sampler = Sampler_Train)
test_embedding_merged = torch.utils.data.DataLoader( AdaptiveDataset(test, path_esm2_test, dir_IFPiFold_Meltome_test, 
                                                    emb_ensemble = 'merged', type_emb='both',
                                                   correspondence_idx = idxdf_idxfile_test) , batch_size=150, shuffle=True,
                                                   batch_sampler = Sampler_Test)
val_embedding_merged = torch.utils.data.DataLoader( AdaptiveDataset(val, path_esm2_val, dir_IFPiFold_Meltome_val, 
                                                    emb_ensemble = 'merged', type_emb='both',
                                                   correspondence_idx = idxdf_idxfile_val) , batch_size=150, shuffle=True,
                                                   batch_sampler = Sampler_Val)
"""

### For RAW ESM2 Using Prediction by Component (convolutions as a way of pooling + sum pooling to get the final predictions)

In [8]:


from src.Trainer_seq_struct_general import Trainer_Seq_Struct_general
from src.loss_functions import loss_manager

#import ipdb; ipdb.set_trace()

#Trainer3 = Trainer_Seq_Struct2(avgtm_mode='global')

from src.Neural_Architectures_general import *


#model_tmp = Encoder_Manager(name='Encoder_LA_MLP_Rank_N_Contrast', dim_embedding= 1280, device= device)
model_tmp = Encoder_Manager(name=config['model_config']['type'], dim_embedding= 1280, device= device)


#optimizers = tmp.set_optimizer()
print('Done')
loss_handler = loss_manager(
    loss=config['loss']['type'],#'rank_n_contrast_loss_representation',
    device=device,    
)
#loss='MSE_loss',

import ipdb; ipdb.set_trace()
print(f'checkpoint pointer to .....{checkpoint_pathESM}')
ESM_only_experiment = Trainer_Seq_Struct_general(
    model_tmp,
    checkpoint_pathESM,
    path_progresESM,
    config['initial_paths']['output_metrics_ESM_only'],
    train_embedding_esm2,
    val_embedding_esm2,
    loss_handler,
    device
)

#import ipdb; ipdb.set_trace()

# Training the model or Loading pretrained models in case of existing
model_to_test = ESM_only_experiment.setup_training()
 

# Testing model
#import ipdb; ipdb.set_trace()
Trainer_Seq_Struct_general.test_model(model_to_test, test_embedding_esm2, device, 
                               f"{config['initial_paths']['output_metrics_ESM_only']}.txt")

#import ipdb; ipdb.set_trace()
del model_tmp
del model_to_test
del loss_handler
del ESM_only_experiment

"Trainer1 = Trainer_Seq_Struct()\nseq_struct_pred_W = tunning_pool_pred_byComp(1280)\n#FFNN_ESM2 = regressionHead(train_embeddings.shape[1::])\noptESM2 = torch.optim.Adam(seq_struct_pred_W.parameters(), lr=1e-4)\nlossESM2= torch.nn.MSELoss()\nepochs = 200\ngenerating_results_approaches(seq_struct_pred_W, Trainer1, train_embedding_esm2, val_embedding_esm2, test_embedding_esm2, \n                              lossESM2, optESM2, epochs, device,\n                              path_progres='train_progress_k1_b10_ESM2.txt', \n                              checkpoint_path='../checkpoints/k1_b10_ESM2/')\n"

### For Inverse Folding PiFold Using Prediction by Component (convolutions as a way of pooling + sum pooling to get the final predictions)

In [9]:
from src.Trainer_seq_struct_general import Trainer_Seq_Struct_general
from src.loss_functions import loss_manager

#import ipdb; ipdb.set_trace()

#Trainer3 = Trainer_Seq_Struct2(avgtm_mode='global')

from src.Neural_Architectures_general import *

#import ipdb; ipdb.set_trace()
#model_tmp = Encoder_Manager(name='Encoder_LA_MLP_Rank_N_Contrast', dim_embedding= 1280, device= device)
model_tmp = Encoder_Manager(name=config['model_config']['type'], dim_embedding= 128, device= device)


#optimizers = tmp.set_optimizer()
print('Done')
loss_handler = loss_manager(
    loss=config['loss']['type'],#'rank_n_contrast_loss_representation',
    device=device,    
)
#loss='MSE_loss',
print(f'checkpoint pointer to .....{checkpoint_pathPiFold}')

PiFold_only_experiment = Trainer_Seq_Struct_general(
    model_tmp,
    checkpoint_pathPiFold,
    path_progresPiFold,
    config['initial_paths']['output_metrics_PiFold_only'],
    train_embedding_PiFold,
    val_embedding_PiFold,
    loss_handler,
    device
)

# Training the model or Loading pretrained models in case of existing
model_to_test = PiFold_only_experiment.setup_training()
 

# Testing model
#import ipdb; ipdb.set_trace()
Trainer_Seq_Struct_general.test_model(model_to_test, test_embedding_PiFold, device, 
                               f"{config['initial_paths']['output_metrics_PiFold_only']}.txt")


del model_tmp
del model_to_test
del loss_handler
del PiFold_only_experiment


"Trainer2 = Trainer_Seq_Struct()\nseq_struct_pred_W_IF = tunning_pool_pred_byComp(128)\n#FFNN_IF = seq_struct_pred\noptIF = torch.optim.Adam(seq_struct_pred_W_IF.parameters(), lr=1e-4)\nlossIF= torch.nn.MSELoss()\nepochs = 200\n\ngenerating_results_approaches(seq_struct_pred_W_IF, Trainer2, train_embedding_PiFold, val_embedding_PiFold, \n                              test_embedding_PiFold,lossIF, optIF, epochs, device,\n                              path_progres='train_progress_k1_b10_PiFold.txt', \n                              checkpoint_path='../checkpoints/k1_b10_PiFold/')\n"

### For Both ESM2 and Inverse Folding PiFold Contributions Together Using Prediction by Component (convolutions as a way of pooling + sum pooling to get the final predictions)

In [ ]:
from src.Trainer_seq_struct_general import Trainer_Seq_Struct_general
from src.loss_functions import loss_manager

#import ipdb; ipdb.set_trace()

#Trainer3 = Trainer_Seq_Struct2(avgtm_mode='global')

from src.Neural_Architectures_general import *


#model_tmp = Encoder_Manager(name='Encoder_LA_MLP_Rank_N_Contrast', dim_embedding= 1280, device= device)
model_tmp = Encoder_Manager(name=config['model_config']['type'], dim_embedding= 1280 + 128, device= device)


#optimizers = tmp.set_optimizer()
print('Done')
loss_handler = loss_manager(
    loss=config['loss']['type'],#'rank_n_contrast_loss_representation',
    device=device,    
)
#loss='MSE_loss',
print(f'checkpoint pointer to .....{checkpoint_pathMix}')

Merge_experiment = Trainer_Seq_Struct_general(
    model_tmp,
    checkpoint_pathMix,
    path_progresMix,
    config['initial_paths']['output_metrics_Both'],
    train_embedding_merged,
    val_embedding_merged,
    loss_handler,
    device
)

# Training the model or Loading pretrained models in case of existing
model_to_test = Merge_experiment.setup_training()
 

# Testing model
#import ipdb; ipdb.set_trace()
Trainer_Seq_Struct_general.test_model(model_to_test, test_embedding_merged, device, 
                               f"{config['initial_paths']['output_metrics_Both']}.txt")



del model_tmp
del model_to_test
del loss_handler
del Merge_experiment 
